# Vox Amelior - Full Stack Colab Setup

This notebook starts **ALL** core services on Google Colab.

**Prerequisites:**
1. Set `TAILSCALE_AUTHKEY` in Colab Secrets (get from Tailscale Admin Console)
2. Set `SESSION_KEY` and `USERS_DB_KEY` (random strings)
3. (Optional) Set `HF_TOKEN` for Gemma models

**Services Started:**
- API Gateway (port 8000)
- Gemma Service (port 8001)
- Transcription Service (port 8003)
- RAG Service (port 8004)
- Emotion Service (port 8005)
- ML Service (port 8006)
- Insights Service (port 8010)
- Fiserv Service (port 8008)

In [ ]:
# @title 1. Configuration
# @markdown Set the branch to pull from GitHub
BRANCH = "colab-dev"  # @param {type:"string"}
REPO_URL = "https://github.com/pruittcolon/Vox_Amelior.git"

# Paths
DRIVE_BASE = "/content/drive/MyDrive/Nemo_Server"
REPO_PATH = f"{DRIVE_BASE}/repo"
LIB_CACHE = f"{DRIVE_BASE}/lib_cache"
DATA_DIR = f"{DRIVE_BASE}/data"

print(f"Branch: {BRANCH}")
print(f"Repo Path: {REPO_PATH}")

In [ ]:
# @title 2. Mount Drive & Clone/Sync Repository
import os
from google.colab import drive

drive.mount('/content/drive')

os.makedirs(LIB_CACHE, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)

if not os.path.exists(os.path.join(REPO_PATH, ".git")):
    print(f"Cloning repository (branch: {BRANCH})...")
    !git clone -b $BRANCH $REPO_URL "$REPO_PATH"
else:
    print(f"Repository found. Syncing with {BRANCH}...")
    %cd $REPO_PATH
    !git fetch origin
    !git checkout $BRANCH
    !git pull origin $BRANCH

%cd $REPO_PATH
print(f"Working directory: {os.getcwd()}")
!git log --oneline -3

In [ ]:
# @title 3. Install System Dependencies
!apt-get update -qq
!apt-get install -y ffmpeg libsndfile1 -qq
print("System dependencies installed.")

In [ ]:
# @title 4. Install Python Dependencies
import os
import sys

if LIB_CACHE not in sys.path:
    sys.path.insert(0, LIB_CACHE)

cache_files = os.listdir(LIB_CACHE) if os.path.exists(LIB_CACHE) else []

if len(cache_files) < 15:
    print("Installing dependencies to Drive cache...")
    !pip install -t "$LIB_CACHE" \
        fastapi uvicorn pydantic httpx nest_asyncio \
        python-multipart python-jose bcrypt cryptography \
        soundfile librosa ffmpeg-python redis faiss-cpu \
        sentence-transformers llama-cpp-python transformers \
        pandas numpy scikit-learn scipy \
        --quiet
    print("Dependencies cached.")
else:
    print(f"Cache found ({len(cache_files)} items).")

In [ ]:
# @title 5. Configure Environment & Monkeypatch
import os
import sys
from google.colab import userdata

# 1. Load secrets
try:
    os.environ["TAILSCALE_AUTHKEY"] = userdata.get('TAILSCALE_AUTHKEY')
    os.environ["SESSION_KEY"] = userdata.get('SESSION_KEY')
    os.environ["USERS_DB_KEY"] = userdata.get('USERS_DB_KEY')
    try: os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
    except: pass
except Exception as e:
    print(f"Warning loading secrets: {e}")

# 2. Service URLs (localhost mode)
os.environ["GEMMA_URL"] = "http://localhost:8001"
os.environ["TRANSCRIPTION_URL"] = "http://localhost:8003"
os.environ["RAG_URL"] = "http://localhost:8004"
os.environ["EMOTION_URL"] = "http://localhost:8005"
os.environ["ML_SERVICE_URL"] = "http://localhost:8006"
os.environ["INSIGHTS_URL"] = "http://localhost:8010"
os.environ["FISERV_URL"] = "http://localhost:8008"

# 3. Paths & Config
os.environ["DB_PATH"] = f"{DATA_DIR}/rag.db"
os.environ["FAISS_INDEX_PATH"] = f"{DATA_DIR}/faiss_index"
os.environ["APP_INSTANCE_DIR"] = f"{DATA_DIR}/instance"
os.environ["TEST_MODE"] = "true"
os.environ["JWT_ONLY"] = "false"

os.makedirs(f"{DATA_DIR}/faiss_index", exist_ok=True)
os.makedirs(f"{DATA_DIR}/instance", exist_ok=True)

# 4. Monkeypatch pysqlcipher3 for Insights Service
import sqlite3
sys.modules['pysqlcipher3'] = sqlite3
sys.modules['pysqlcipher3.dbapi2'] = sqlite3

print("Environment configured & patched.")

In [ ]:
# @title 6. Start ALL Services
import threading
import time
import sys
import os
import uvicorn
import nest_asyncio

nest_asyncio.apply()

# Update path for imports
sys.path.insert(0, REPO_PATH)
for service in ["api-gateway", "gemma-service", "transcription-service", 
                "rag-service", "emotion-service", "ml-service", 
                "insights-service", "fiserv-service"]:
    sys.path.insert(0, os.path.join(REPO_PATH, f"services/{service}"))

# Generic launcher
def start_service(name, import_path, port):
    print(f"Starting {name} on port {port}...")
    try:
        uvicorn.run(import_path, host="0.0.0.0", port=port, log_level="error")
    except Exception as e:
        print(f"{name} error: {e}")

# Thread launcher
services = [
    ("API Gateway", "services.api-gateway.src.main:app", 8000),
    ("Gemma", "services.gemma-service.src.main:app", 8001),
    ("Transcription", "services.transcription-service.src.main:app", 8003),
    ("RAG", "services.rag-service.src.main:app", 8004),
    ("Emotion", "services.emotion-service.src.main:app", 8005),
    ("ML", "services.ml-service.src.main:app", 8006),
    ("Fiserv", "services.fiserv-service.src.main:app", 8008),
    ("Insights", "services.insights-service.src.main:app", 8010),
]

threads = []
for name, path, port in services:
    t = threading.Thread(target=start_service, args=(name, path, port), daemon=True)
    t.start()
    time.sleep(2)  # Stagger to be nice to CPU

print("\nAll services requested. Waiting 15s for initialization...")
time.sleep(15)
print("Services should be ready.")

In [ ]:
# @title 7. Setup Tailscale (Unlimited Bandwidth)
import os
import subprocess

# Install Tailscale
print("Installing Tailscale...")
!curl -fsSL https://tailscale.com/install.sh | sh

# Start Tailscale in userspace mode (required for Colab)
AUTHKEY = os.environ.get("TAILSCALE_AUTHKEY", "")
if not AUTHKEY:
    print("ERROR: Set TAILSCALE_AUTHKEY in Colab Secrets!")
    print("Get it from: https://login.tailscale.com/admin/settings/keys")
else:
    print("Starting Tailscale...")
    # Run in background with userspace networking
    !sudo tailscaled --tun=userspace-networking --state=/tmp/tailscale.state &
    import time
    time.sleep(3)
    !sudo tailscale up --authkey=$AUTHKEY --hostname=vox-amelior-colab
    
    # Get the Tailscale IP
    result = subprocess.run(["tailscale", "ip", "-4"], capture_output=True, text=True)
    tailscale_ip = result.stdout.strip()
    
    print("\n" + "="*50)
    print("TAILSCALE CONNECTED")
    print("="*50)
    print(f"Access URL: http://{tailscale_ip}:8000/ui/login.html")
    print(f"API Base:   http://{tailscale_ip}:8000")
    print("="*50)
    print("\nNOTE: Access only from devices on your Tailscale network.")

In [ ]:
# @title 8. Health Check
import requests
print("Checking Services:")
ports = {
    8000: "API Gateway",
    8001: "Gemma",
    8003: "Transcription",
    8004: "RAG",
    8005: "Emotion",
    8006: "ML",
    8008: "Fiserv",
    8010: "Insights"
}
for port, name in ports.items():
    try:
        r = requests.get(f"http://localhost:{port}/health", timeout=2)
        status = "UP" if r.status_code == 200 else f"Code {r.status_code}"
    except:
        status = "DOWN"
    print(f"  {name:15} (:{port}): {status}")